In [ ]:
!pip install apyori

In [ ]:
import pandas as pd
import numpy as np
from apyori import apriori

In [ ]:
df = pd.read_csv('DATA.csv', nrows = 100)

In [ ]:
df

In [ ]:
df = df.drop(['39','77516','13','2174','0','40'],axis = 1)

In [ ]:
df

In [ ]:
records = []
for i in range(0, 100):
    records.append([str(df.values[i,u]) for u in range(0, 9)])

In [ ]:
rules = apriori(records, min_support=0.0022, min_confidence=0.20, min_lift=3, min_length = 4)

In [ ]:
# Store rules in result variable
results = list(rules)# See the items that were bought together with their support
results_list = []
for i in range(0, len(results)):
    results_list.append('RULE:\t' + str(results[i][0]) + '\nSUPPORT:\t' + str(results[i][1]))

In [ ]:
print(len(results_list))

In [ ]:
print(results_list)

RULE 1:\tfro
zenset({'Instant food products', 'hamburger meat'})\nSUPPORT:\t0.003050330452465684

If a customer buys "Instant food products" and "hamburger meat", then there is a support of 0.00305, which means that these items are purchased together in approximately 0.305% of all transactions.

In [ ]:
columns=["age","workclass","fnlwgt","education","education-num","marital-status","occupation","relationship","race","sex","capital-gain","capital-loss","hours-per-week","native-country","income"]
df = pd.read_csv('DATA.csv',header=None,names=columns)
df.head()

df.isnull().sum()

df.columns

num_cols = df.select_dtypes(include=['int64', 'float64', 'int32']).columns
num_cols

# One-Hot Encoding
from sklearn.preprocessing import OneHotEncoder

categorical_cols = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']

# Create a new DataFrame with only relevant columns
categorical_df = df[categorical_cols]
num_df=df[num_cols]
encoder = OneHotEncoder()

# Fit and transform the categorical data
encoded_data = encoder.fit_transform(categorical_df).toarray()

# Create a DataFrame from the encoded data
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(categorical_cols))

# Combine the encoded DataFrame with the original numeric columns
binarized_df = pd.concat([encoded_df], axis=1)

# Display the combined DataFrame
print(binarized_df.head())

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# Set your support and confidence thresholds
min_support_values = [0.05, 0.1, 0.15]
min_confidence = 0.7

# Calculate frequent itemsets
for min_support in min_support_values:
    frequent_itemsets = apriori(binarized_df, min_support=min_support, use_colnames=True)

    # Extract association rules
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_confidence)

    # Rank the top 10 rules
    top_10_rules = rules.nlargest(10, "confidence")

    # Print rules and explanations
    print(f"Rules for min_support={min_support:.2f}:")
    for i, (_, row) in enumerate(top_10_rules.iterrows(), start=1):
        antecedent = row['antecedents']
        consequent = row['consequents']
        confidence = row['confidence']

        print(f"Rule {i}: If {antecedent} then {consequent} (Confidence: {confidence:.2f})")

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

def calculate_interest(rule):
    return rule['confidence'] * rule['lift']

min_support_values = [0.05, 0.1, 0.15]
min_interest_values = [0.5, 0.7, 0.9]

for min_support in min_support_values:
    frequent_itemsets = apriori(binarized_df, min_support=min_support, use_colnames=True)

    for min_interest in min_interest_values:
        rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_confidence)

        # Calculate interest measure for each rule
        rules['interest'] = rules.apply(calculate_interest, axis=1)

        # Rank the top 10 rules based on interest
        top_10_rules = rules.nlargest(10, "interest")

        # Print rules and explanations
        print(f"Rules for min_support={min_support:.2f} and min_interest={min_interest:.2f}:")
        for i, (_, row) in enumerate(top_10_rules.iterrows(), start=1):
            antecedent = row['antecedents']
            consequent = row['consequents']
            confidence = row['confidence']
            lift = row['lift']
            interest = row['interest']

            print(f"Rule {i}: If {antecedent} then {consequent} (Confidence: {confidence:.2f}, Lift: {lift:.2f}, Interest: {interest:.2f})")

